# Progetto di Social Computing

a.a. 2022/2023

## Attività preliminari

### Librerie e costanti

In [1]:
# Import delle librerie utilizzate
import os, tweepy, json
import networkx as nx 
import pandas as pd 
import numpy as np 
import random as rn
import networkx as nx
import matplotlib.pyplot as plt 

In [2]:
# Cartelle di salvataggio
data_folder = "./data"
out_folder = "./out"

### Funzioni

In [3]:
# Salvataggio in locale
def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent = 4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

In [4]:
# Lettura da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [16]:
# Somma dei numeri di tweet prodotti in un intervallo di tempo
def sum_tweets_count(tweet_groups):
    sum = 0

    for tweet_count in tweet_groups:
        sum += tweet_count["tweet_count"]
    
    return sum

In [21]:
def normalize_into(user, root_distance, destination):
    normalized = {
        "name" : user["name"],
        "username" : user["username"],
        "description" : user["description"],
        "public_metrics" : user["public_metrics"],
        "protected" : user["protected"]
    }

    # Se user non è protetto, si possono aggiungere i dettagli aggiuntivi
    if (root_distance < 2 and not user["protected"]):
        # Si estraggono le ids dei follower di 'follower'
        f_ids = []
        for f in user["followers"]:
            f_ids.append(f["id"])
    
        normalized["last_week_tweets_count"] = user["last_week_tweets_count"]
        normalized["followers"] = f_ids
    
    # Non si aggiorna se l'utente è già definito
    if user["id"] not in destination:
        destination[user["id"]] = normalized

### Credenziali Twitter API

In [6]:
# Caricamento credenziali da JSON
api_access = read_json("./api_access.json")

Data read from path: ./api_access.json


## Recupero dei follower e dei follower dei follower

### Recupero dei follower

Si vogliono recuperare, utilizzando la libreria `tweepy`, tutti i follower dell'utente *@KevinRoitero*, corredati delle seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

In [18]:
 # Si inizializza il client
client = tweepy.Client(bearer_token=api_access["bearer_token"])

username = "KevinRoitero"
all_user_followers = []

# Si recuperano e memorizzano le informazioni dell'utente
response = client.get_user(username = username, user_fields=["description", "protected", "public_metrics"])
user = dict(response.data)

# Si recuperano e memorizzano i follower dell'utente
response = client.get_users_followers(user["id"], user_fields=["description", "protected", "public_metrics"], 
                                      max_results=150) # max_results = 150 perché i follower dell'utente sono nell'ordine di 130
for follower in response.data:
    all_user_followers.append(dict(follower))

# Si associano i follower trovati all'utente di partenza
user["followers"] = all_user_followers

# Si serializza su JSON il risultato ottenuto
serialize_json(data_folder, "user_followers.json", user)

Data serialized to path: ./data/user_followers.json


### Aggiunta del numero di tweet prodotti nell'ultima settimana

Ai follower trovati, si vuole aggiungere il numero di tweet pubblicati nell'ultima settimana. Per avere uniformità, troveremo anche il numero di tweet pubblicati nell'ultima settimana dall'utente *@KevinRoitero*.

In [19]:
# Si sceglie l'utilizzo dello stesso client di partenza, o comunque senza la possibilità di mettersi in attesa, 
# perché numero_richieste < 300, dove 300 è il numero massimo di richieste per l'endpoint get_recent_tweets_count()
client = tweepy.Client(bearer_token=api_access["bearer_token"])

# Si caricano i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/user_followers.json")

# Si memorizza il numero di tweet pubblicati dall'utente nell'ultima settimana
response = client.get_recent_tweets_count(query="from:"+user["username"], granularity="day")
user["last_week_tweets_count"] = sum_tweets_count(response.data)

# Si ripete lo stesso per i follower dell'utente
for follower in user["followers"]:
    if(not follower["protected"]): # non si può accedere ai tweet di un utente 'protected'
        response = client.get_recent_tweets_count(query="from:"+follower["username"], granularity="day")
        follower["last_week_tweets_count"] = sum_tweets_count(response.data)

# Si serializza su JSON il risultato ottenuto
serialize_json(data_folder, "followers_last_week_tweets.json", user)

Data read from path: ./data/user_followers.json
Data serialized to path: ./data/followers_last_week_tweets.json


### Recupero dei follower dei follower

Per ciascun follower di *@KevinRoitero* avente almeno 1 follower e non `protected`, si vogliono scaricare le seguenti informazioni:

* attributi di default;
* descrizione del profilo;
* metriche pubbliche dell'account;
* se l'account è protetto

L'approccio che è stato seguito è quello di un download *parallelo*, ossia ciascun collaboratore al progetto si è preso in carico di scaricare una porzione di follower dei follower di *@KevinRoitero*.

In [7]:
# Si impone al client di attendere nel caso di raggiungimento del limite delle richieste
client = tweepy.Client(bearer_token=api_access["bearer_token"], wait_on_rate_limit=True)

# Si caricano i dati dell'utente e quelli dei suoi follower
user = read_json(data_folder+"/followers_last_week_tweets.json")

# Parallelizzando il calcolo, si decide di partizionare tra collaboratori il numero di follower
all_followers = np.array(user["followers"])

f_num = user["public_metrics"]["followers_count"]
start = f_num*api_access["id"] // 4
end = f_num*(api_access["id"]+1) // 4

# Si vogliono memorizzare solamente i follower di responsabilità del collaboratore
interval_followers = []

# Si scaricano i follower dei follower nell'intervallo del collaboratore
for i in range(start, end):
    # Si considera l'i-esimo follower
    follower = all_followers[i]

    # Se l'i-esimo follower non è protetto e ha dei follower, li scarico
    if (not follower["protected"] and follower["public_metrics"]["followers_count"] > 0):
        for all_follower_followers in tweepy.Paginator(client.get_users_followers, id = follower["id"], user_fields=["description", "protected", "public_metrics"], max_results = 1000):
            parsed_followers = []
            
            for ff in all_follower_followers.data:
                # L'oggetto User deve essere interpretato
                parsed_follower = {
                    "id" : ff["id"],
                    "public_metrics" : ff["public_metrics"],
                    "description" : ff["description"],
                    "name" : ff["name"],
                    "protected" : ff["protected"],
                    "username" : ff["username"]
                }
                parsed_followers.append(parsed_follower)
            
            follower["followers"] += parsed_followers
    
    # Aggiungo il follower (arricchito dei suoi follower) nella lista di follower di competenza del collaboratore
    interval_followers.append(follower)

# Si salva la porzione di follower scaricata
serialize_json(data_folder, "f_of_f_"+str(api_access["id"])+".json", list(interval_followers))

Data read from path: ./data/followers_last_week_tweets.json


Rate limit exceeded. Sleeping for 889 seconds.
Rate limit exceeded. Sleeping for 893 seconds.
Rate limit exceeded. Sleeping for 891 seconds.


Data serialized to path: ./data/f_of_f_2


Si uniscono ora i file generati dai diversi collaboratori in un unico file JSON.

In [15]:
# Si carica il JSON con le informazioni su @KevinRoitero
user = read_json(data_folder+"/followers_last_week_tweets.json")
followers = []

for i in range(0, 4):
    # Si caricano i follower individuati dal collaboratore i
    partial_followers = read_json(data_folder+"/f_of_f_"+str(i)+".json")
    
    # Si concatenano all'elenco completo dei follower
    followers += partial_followers

# Si aggiornano i dati sui follower di @KevinRoitero
user["followers"] = followers

# Si serializza il risultato ottenuto
serialize_json(data_folder, "user_complete.json", user)

Data read from path: ./data/followers_last_week_tweets.json
Data read from path: ./data/f_of_f_0.json
Data read from path: ./data/f_of_f_1.json
Data read from path: ./data/f_of_f_2.json
Data read from path: ./data/f_of_f_3.json
Data serialized to path: ./data/user_complete.json


### Riformattazione del JSON finale

Al fine di essere conformi alle specifiche della consegna, si decide di ristrutturare (ossia modificarne la presentazione, pur mantenendo invariate le informazioni al suo interno) il JSON `user_complete.json` definito in precedenza.

Il formato che è stato ritenuto come più opportuno, sia per un'efficienza di archiviazione che di computazione (importante per le sezione successive), è il seguente:
```json
{
    ...
    id (int) : {
        "name" : str,
        "username" : str,
        "description" : int,
        "public_metrics" : {
            "followers_count" : int,
            "following_count" : int,
            "tweet_count" : int,
            "listed_count" : int
        },
        "protected" : bool,
        ("last_week_tweets_count" :int,)
        ("followers" : [
            ...
            f_id (int),
            ...
        ])
    }
    ...
}
```

Dove `id` rappresenta l'identificativo univoco di un utente, le cui informazioni sono state scaricate nei passi precedenti, ed `f_id` rappresenta l'identificativo di un follower di `id`.

È doveroso far notare che, per come sono stati scaricati gli utenti, solamente coloro che o sono *@KevinRoitero* o sono suoi follower (non `protected`) presentano il campo `last_week_tweets_count` ed il campo `followers` (da cui le parentesi tonde nello schema).

In [22]:
# Si carica il JSON completo, non formattato
root_user = read_json(data_folder+"/user_complete.json")

nodes = {} # JSON finale

normalize_into(user = root_user, root_distance=0, destination = nodes)

# Si fa la stessa cosa per follower di @KevinRoitero
for follower in root_user["followers"]:
    normalize_into(user = follower, root_distance = 1, destination = nodes)

# ... e per i follower dei follower
for follower in root_user["followers"]:
    if (not follower["protected"]):
        for ff in follower["followers"]:
            normalize_into(user = ff, root_distance = 2, destination = nodes)

# Si serializza il JSON ben formattato
serialize_json(data_folder, "users_final.json", nodes)

Data read from path: ./data/user_complete.json
Data serialized to path: ./data/users_final.json


Si consideri che, per come è stato costruito, il file `users_final.json` presenta una struttura ordinata:

1. Al primo posto vi sonoi dettagli sul profilo di *@KevinRoitero*;
2. dal secondo posto fino al 134-esimo vi sono i follower di *@KevinRoitero*;
3. dal 135-esimo posto in poi vi sono i follower dei follower di *@KevinRoitero*.

## Creazione della rete sociale diretta

### Aggiunta dei nodi

Come nodi si vogliono avere *@KevinRoitero* ed i suoi follower ed ogni nodo deve rispettare le seguenti caratteristiche:

* il suo `id` deve essere uguale all'`id` del profilo utente;
* deve avere come attributi:
    * lo username;
    * la descrizione;
    * il numero di follower del profilo.

In [23]:
# Si crea un grafo diretto vuoto
social_graph = nx.DiGraph()

# Si considera l'insieme dei profili utente
users = read_json(data_folder+"/users_final.json")

# La struttura del file permette di fare un inserimento "ordinato"
added = 0
for id in users:
    user = users[id]
    social_graph.add_node(int(id), username = user["username"], 
                              description = user["description"], 
                              followers_count = user["public_metrics"]["followers_count"])
    
    added += 1
    # Il 134-esimo utente è l'ultimo follower di @KevinRoitero
    if(added == 134):
        break

# N.B.: questo trucchetto dell'inserimento ordinato è possibile solamente grazie al fatto che la funzione read_json() ed il costrutto for,
#       rispettivamente, caricano in memoria principale le informazioni in ordine di apparizione ed iterano su di esse rispettando ancora
#       il loro ordine di apparizione. Se questi dettagli implementativi dovessero cambiare, sarebbe necessario apportare modifiche anche
#       a questa soluzione.

Data read from path: ./data/users_final.json


### Aggiunta degli archi

Al grafo definito precedntemente si vogliono aggiungere archi $(v,w)$ tra due nodi $v$ e $w$ se e solo se il profilo corrispondente a $v$ è follower del profilo corrispondente a $w$.

In [26]:
# Si ereditano le variabili 'social_graph' e 'users' dal chunk precedente
for v in social_graph.nodes:
    for w in social_graph.nodes:
        if ((not v == w) and (not users[str(w)]["protected"]) and (v in users[str(w)]["followers"])):
            social_graph.add_edge(v, w)